In [13]:
import rocksdb
import numpy as np

In [14]:
opts = rocksdb.Options(create_if_missing=True)
db = rocksdb.DB("YourDatabase.db",opts,read_only=False) 

In [15]:
# Create Column families for inputs and labels 
# Inputs
opts_inputs = rocksdb.ColumnFamilyOptions()
db.create_column_family(b'inputs',opts_inputs)

# labels
opts_labels = rocksdb.ColumnFamilyOptions()
db.create_column_family(b'labels',opts_labels)

<ColumnFamilyHandle name: b'labels', id: 2, state: valid>

In [16]:
# Now the standard machinery applies for storing and retrieving keys 
img_shape   = (3,128,128)
img_dtype   = np.float32
label_shape = (1,128,128)
label_dtype = np.uint8


In [17]:
# datum_index=123
# But we only need one key as we are storing images and labels in different column families
key   = b'123'
timg      = np.random.rand(*img_shape).astype(img_dtype)
tlabel    = np.random.rand(*label_shape).astype(label_dtype) # 1hot encoding

In [18]:
cf_in = db.get_column_family(b'inputs')
db.put((cf_in,key),timg.tobytes())

In [19]:
cf_label = db.get_column_family(b'labels')
db.put((cf_label,key),tlabel.tobytes())

In [21]:
# Now read back: 
cf_in    = db.get_column_family(b'inputs')
cf_labels = db.get_column_family(b'labels')
timg_retrieved   = np.frombuffer(db.get((cf_in,key)),dtype=img_dtype).reshape(*img_shape)
tlabel_retrieved = np.frombuffer(db.get((cf_labels,key)),dtype=label_dtype).reshape(*label_shape)

In [22]:
(timg_retrieved==timg).all()

True

In [23]:
(tlabel_retrieved==tlabel).all()

True

In [24]:
db.close()